# Plot 1: RD plots for avgs in a dataset and some images

In [ ]:
from typing import Literal

import matplotlib.pyplot as plt

from coolchic.eval.hypernet import plot_hypernet_rd, plot_hypernet_rd_avg
from coolchic.eval.results import parse_hypernet_metrics
from coolchic.utils.paths import DATA_DIR, RESULTS_DIR

plt.rcParams["figure.dpi"] = 300

In [ ]:
sweep_path = RESULTS_DIR / "exps/copied/delta-hn/from-orange/"
compare_no_path = RESULTS_DIR / "exps/copied/no-cchic/orange-nocc/"

In [ ]:
def rd_plots_from_dataset(dataset: Literal["kodak", "clic20-pro-valid"]) -> None:
    import pandas as pd

    metrics = parse_hypernet_metrics(sweep_path, dataset=dataset, premature=True)
    df = pd.concat(
        [
            pd.DataFrame(
                [s.model_dump() for seq_res in metrics.values() for s in seq_res]
            ).assign(anchor="hypernet (ours)"),
            pd.DataFrame(  # For comparison with no hypernet.
                [
                    s.model_dump()
                    for seq_res in parse_hypernet_metrics(
                        compare_no_path, dataset, premature=True
                    ).values()
                    for s in seq_res
                ]
            ).assign(anchor="n-o cool-chic"),
        ],
    ).sort_values(by=["seq_name", "lmbda"])  # So plots come out nice and in order.

    plot_hypernet_rd_avg(df, dataset=dataset)

    all_images = sorted(list((DATA_DIR / dataset).glob("*.png")), key=lambda x: x.stem)
    for img in all_images[:5]:
        seq_name = img.stem
        plot_hypernet_rd(seq_name, df, dataset=dataset)

In [ ]:
rd_plots_from_dataset("kodak")

In [ ]:
rd_plots_from_dataset("clic20-pro-valid")

# Plot 2: flops vs BD-rate

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from coolchic.eval.hypernet import get_hypernet_flops
from coolchic.hypernet.hypernet import DeltaWholeNet, NOWholeNet

sns.set_theme(context="notebook", style="whitegrid")

In [ ]:
flops = {
    "coolchic": (
        coolchic_fwd := get_hypernet_flops(DeltaWholeNet, get_coolchic_flops=True)
    ),
    "coolchic_step": 3 * coolchic_fwd,
    "nocc": get_hypernet_flops(NOWholeNet),
    "delta": get_hypernet_flops(DeltaWholeNet),
}

bd_rates = {
    "kodak": {
        "N-O Cool-chic": 37.33,
        "Hypernet (ours)": 33.60,
    },
    "clic20-pro-valid": {
        "N-O Cool-chic": 36.233904571367106,
        "Hypernet (ours)": 28.284958907776783,
    },
}

metrics_dfg = pd.DataFrame(
    [
        (dataset, method, bd_rate, flops[method])
        for dataset, bd_rates in bd_rates.items()
        for method, bd_rate in bd_rates.items()
    ],
    columns=["dataset", "method", "bd_rate", "flops"],
)
metrics_dfg["num_coolchic_steps"] = metrics_dfg["flops"] / flops["coolchic_step"]
metrics_dfg.head()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    data=metrics_dfg[metrics_dfg["dataset"] == "clic20-pro-valid"],
    x="num_coolchic_steps",
    y="bd_rate",
    hue="method",
    ax=ax,
)
ax.set_title("BD Rate vs. Equivalent number of Cool-chic Steps, CLIC20 dataset")
sns.despine()

# Plot 3: finetuning

In [ ]:
from collections import defaultdict

import pandas as pd

from coolchic.eval.hypernet import find_crossing_it
from coolchic.utils.paths import COOLCHIC_REPO_ROOT

In [ ]:
finetuning_data = pd.read_csv(COOLCHIC_REPO_ROOT / "clic_finetuning_results.csv")
dataset = "clic20-pro-valid"
freq_valid = 10  # Assuming the finetunig_results file was generated with our code.

crossing_its: dict[str, dict[str, list[dict[Literal["hn", "scratch"], int]]]] = {
    "jpeg": defaultdict(list),
    "hm": defaultdict(list),
    "hypernet": defaultdict(list),
}
for image in (DATA_DIR / dataset).glob("*.png"):
    # Skip images if they are not in the results.
    if image.stem not in finetuning_data["seq_name"].values:
        continue
    plot_hypernet_rd(image.stem, finetuning_data, dataset)
    for anchor_name in crossing_its:
        crossing_its[anchor_name][image.stem].append(
            {
                "hn": find_crossing_it(
                    image.stem,
                    finetuning_data,
                    "nocc-finetuning",
                    anchor_name=anchor_name,
                    dataset=dataset,
                ),
                "scratch": find_crossing_it(
                    image.stem,
                    finetuning_data,
                    "coolchic-training",
                    anchor_name=anchor_name,
                    dataset=dataset,
                ),
            }
        )

for anchor_name, crossings_per_img in crossing_its.items():
    print(f"Crossing iterations for {anchor_name}")
    for seq_name, crossings in crossings_per_img.items():
        for cross in crossings:
            print(
                f"{seq_name:<30}, crossing iterations: "
                f"nocc-finetuning: {cross['hn'] * freq_valid if cross['hn'] > 0 else 'no':>4}, "
                f"coolchic-training: {cross['scratch'] * freq_valid if cross['scratch'] > 0 else 'no':>4}"
            )
# plt.show()

# Plot 4: show bd rate vs flops as training progresses

In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from coolchic.eval.bd_rate import bd_rates_summary_anchor_name
from coolchic.eval.hypernet import get_hypernet_flops
from coolchic.eval.results import SummaryEncodingMetrics
from coolchic.hypernet.hypernet import DeltaWholeNet, NOWholeNet
from coolchic.utils.paths import RESULTS_DIR

sns.set_theme(context="notebook", style="whitegrid")
plt.rcParams["figure.dpi"] = 600

In [ ]:
flops = {
    "coolchic": (
        coolchic_fwd := get_hypernet_flops(DeltaWholeNet, get_coolchic_flops=True)
    ),
    "coolchic_step": 3 * coolchic_fwd,
    "N-O Cool-chic": get_hypernet_flops(NOWholeNet),
    "Hypernet (ours)": get_hypernet_flops(DeltaWholeNet),
}

bd_rates = {
    "kodak": {
        "N-O Cool-chic": 37.33,
        "Hypernet (ours)": 33.60,
    },
    "clic20-pro-valid": {
        "N-O Cool-chic": 36.233904571367106,
        "Hypernet (ours)": 28.284958907776783,
    },
}


metrics_df = pd.DataFrame(
    [
        (dataset, method, bd_rate, flops[method])
        for dataset, bd_rates in bd_rates.items()
        for method, bd_rate in bd_rates.items()
    ],
    columns=["dataset", "method", "bd_rate", "flops"],
)
metrics_df["num_coolchic_steps"] = metrics_df["flops"] / flops["coolchic_step"]
metrics_df.head()

In [ ]:
# dataset = "clic20-pro-valid"
dataset = "kodak"
method = "from-orange"
CONFIG_NUM_TO_LMBDA = {"00": 0.0001, "01": 0.0004, "02": 0.001, "03": 0.004, "04": 0.02}

finetuning_dir = RESULTS_DIR / "finetuning" / dataset
finetuning_df = pd.concat(
    [
        pd.read_csv(file).assign(anchor=f"{file.parent.stem}-finetuning")
        for file in finetuning_dir.glob("**/*.csv")
    ]
)
finetuning_df = finetuning_df.reset_index(drop=True)
finetuning_df["anchor"] = finetuning_df["anchor"].apply(
    lambda x: "N-O Cool-chic finetuning" if "nocc" in x else "Hypernet finetuning"
)

In [ ]:
def get_bd_rate_from_df(df: pd.DataFrame) -> float:
    metrics = defaultdict(list)
    for _, row in df.iterrows():
        assert "seq_name" in row, "seq_name column is missing in the DataFrame"
        metrics[row["seq_name"]].append(
            SummaryEncodingMetrics(
                seq_name=row["seq_name"],
                lmbda=row["lmbda"],
                rate_bpp=row["rate_bpp"],
                psnr_db=row["psnr_db"],
            )
        )

    bds = bd_rates_summary_anchor_name(
        metrics, anchor="hm", dataset=dataset, only_latent_rate=False
    )
    assert len(bds) == 1, "Expected exactly one BD rate result."
    return list(bds.values())[0]


bd_df = (
    finetuning_df.groupby(["n_itr", "seq_name", "anchor"], group_keys=False)
    .apply(get_bd_rate_from_df)
    .reset_index()
    .rename(columns={0: "bd_rate"})
)
bd_df["num_coolchic_steps"] = (
    bd_df["n_itr"] + flops["N-O Cool-chic"] / flops["coolchic_step"]
)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(
    data=metrics_df[metrics_df["dataset"] == dataset],
    x="num_coolchic_steps",
    y="bd_rate",
    hue="method",
    ax=ax,
)
avg_bd_df = (
    bd_df.groupby(["anchor", "n_itr"])
    .agg({"bd_rate": "mean", "num_coolchic_steps": "mean"})
    .reset_index()
)
# Add the BD rates from the init points to give an impression of finetuning from the start.
avg_bd_df = pd.concat(
    [
        avg_bd_df,
        metrics_df[metrics_df["dataset"] == dataset][
            ["num_coolchic_steps", "bd_rate", "method"]
        ]
        .rename(columns={"method": "anchor"})
        .assign(
            anchor=lambda df: df["anchor"]
            .apply(lambda x: x.strip(" (ours)"))
            .apply(lambda x: f"{x} finetuning")
        )
        .assign(n_itr=0),
    ]
)
# Need this to match the hue order with the scatter plot.
methods_used = [
    f"{name.strip(' (ours)')} finetuning" for name in metrics_df.method.unique()
]
sns.lineplot(
    data=avg_bd_df,
    x="num_coolchic_steps",
    y="bd_rate",
    hue="anchor",
    hue_order=methods_used,
    ax=ax,
)
ax.set_title(
    f"BD Rate vs. Equivalent number of Cool-chic Steps, averaged over {'CLIC2020' if dataset == 'clic20-pro-valid' else 'Kodak' if dataset == 'kodak' else None} dataset"
)
ax.set_xlabel("Equivalent number of Cool-chic steps")
ax.set_ylabel("average BD Rate (vs. HEVC)")
sns.despine()

In [ ]:
# MACs per pixel of a Cool-chic training step.
flops["coolchic_step"] / (512**2)

In [ ]:
# MACs per pixel of a Cool-chic forward pass.
{name: flops[name] / (512**2) for name in flops}

# Plot 5: extra rate and benefits of adding each part

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch

# from coolchic.eval.results import parse_hypernet_metrics
from coolchic.utils.paths import RESULTS_DIR

plt.rcParams["figure.dpi"] = 300

In [ ]:
dataset = "kodak"
baseline_metrics = pd.DataFrame(
    [
        s.model_dump()
        for seq_res in parse_hypernet_metrics(
            RESULTS_DIR / "exps/copied/no-cchic/orange-nocc/",
            dataset=dataset,
            premature=True,
        ).values()
        for s in seq_res
    ]
).assign(variant="baseline")
nosynth_metrics = pd.DataFrame(
    [
        s.model_dump()
        for seq_res in parse_hypernet_metrics(
            RESULTS_DIR / "eval_no_synth", dataset=dataset, premature=True
        ).values()
        for s in seq_res
    ]
).assign(variant="with ARM modulations")
noarm_metrics = pd.DataFrame(
    [
        s.model_dump()
        for seq_res in parse_hypernet_metrics(
            RESULTS_DIR / "eval_no_arm", dataset=dataset, premature=True
        ).values()
        for s in seq_res
    ]
).assign(variant="with synthesis modulations")
both_metrics = pd.DataFrame(
    [
        s.model_dump()
        for seq_res in parse_hypernet_metrics(
            RESULTS_DIR / "exps/copied/delta-hn/from-orange/",
            dataset=dataset,
            premature=True,
        ).values()
        for s in seq_res
    ]
).assign(variant="full hypernetwork (ours)")
all_metrics = pd.concat(
    [baseline_metrics, nosynth_metrics, noarm_metrics, both_metrics],
    ignore_index=True,
).sort_values(by=["seq_name", "lmbda"])

In [ ]:
all_metrics.head()

In [ ]:
diffs = (
    all_metrics.groupby("variant")
    .agg(
        {
            "rate_nn_bpp": "mean",
            "psnr_db": "mean",
            "rate_latent_bpp": "mean",
        }
    )
    .reindex(
        [
            "baseline",
            "with ARM modulations",
            "with synthesis modulations",
            "full hypernetwork (ours)",
        ]
    )
)
baseline_values = diffs.iloc[0]
diffs = diffs - baseline_values
diffs = diffs.reset_index()

# Invert the bpp metrics so that an improvement (decrease) is positive
diffs = diffs.rename(
    columns={
        "rate_nn_bpp": "rate_nn_bpp_diff",
        "psnr_db": "psnr_db_diff",
        "rate_latent_bpp": "rate_latent_bpp_diff",
    }
)
diffs

In [ ]:
# Melt the DataFrame for seaborn
plot_data = diffs.melt(
    id_vars=["variant"],
    value_vars=["rate_nn_bpp_diff", "rate_latent_bpp_diff", "psnr_db_diff"],
    var_name="metric",
    value_name="value",
)

# Separate data for the two axes
bpp_data = plot_data[plot_data["metric"].str.contains("bpp")]
psnr_data = plot_data[plot_data["metric"].str.contains("psnr")]

# Pivot the data and remove the baseline variant (its diff is always 0)
diffs_pivot = diffs.set_index("variant").loc[
    ["with ARM modulations", "with synthesis modulations", "full hypernetwork (ours)"]
]

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 7))

# Plot BPP data on the first y-axis (ax1)
sns.barplot(
    data=bpp_data,
    x="variant",
    y="value",
    hue="metric",
    ax=ax1,
    palette="viridis_r",  # Use a distinct color palette
)

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()

# Plot PSNR data on the second y-axis (ax2)
# We plot it as a point plot to avoid overlapping bars, but a bar plot can also work
sns.pointplot(
    data=psnr_data,
    x="variant",
    y="value",
    hue="metric",
    ax=ax2,
    palette=["red"],  # Use a single, distinct color
    markers="D",  # Use a diamond marker
    linestyles="--",  # Use a dashed line
)

# Set titles and labels for the first axis (BPP)
ax1.set_title("Model Ablation Study: Metric Changes from Baseline", fontsize=16)
ax1.set_xlabel("Model Variant", fontsize=12)
ax1.set_ylabel("$\Delta$ BPP (Lower is Better)", fontsize=12)  # <-- Key change here
ax1.axhline(0, color="gray", linewidth=0.8, linestyle="--")

# Set label for the second axis (PSNR)
ax2.set_ylabel("$\Delta$ PSNR (dB) (Higher is Better)", fontsize=12, color="red")
ax2.tick_params(axis="y", labelcolor="red")
# Ensure the zero-point of the second axis aligns with the first
ax2.axhline(0, color="gray", linewidth=0.8, linestyle="--")


# Customize Legends
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

# Create new labels for clarity
new_labels1 = ["$\Delta$ Rate NN (bpp)", "$\Delta$ Rate Latent (bpp)"]
new_labels2 = ["$\Delta$ PSNR (dB)"]

# Combine legends
ax1.legend(
    handles1, new_labels1, title="Rate Metrics (Lower is better)", loc="upper left"
)
ax2.legend(
    handles2,
    new_labels2,
    title="Distortion Metric (Higher is better)",
    loc="upper right",
)

plt.tight_layout()
plt.show()

In [ ]:
# --- 4. Create the Plot ---
fig, ax1 = plt.subplots(figsize=(6, 4))
ax2 = ax1.twinx()

# --- Bar properties ---
variants = diffs_pivot.index
x_pos = np.arange(len(variants))  # The label locations [0, 1, 2]
bar_width = 0.20  # Set the desired width of the bars
offsets = np.array([-bar_width, 0, bar_width])  # Offsets for the three bars

# Define colors
# Dutch field color palette: https://gist.github.com/afcotroneo/ca9716f755128b5e9b2ed1fe4186f4df#file-omni-dutch-field
bpp1_color = "#0bb4ff"
bpp2_color = "#50e991"
psnr_color = "#e60049"  # A slightly different red

# --- Plotting the bars manually ---
# Plot BPP bars on ax1
ax1.bar(
    x_pos + offsets[0], diffs_pivot["rate_nn_bpp_diff"], bar_width, color=bpp1_color
)
ax1.bar(
    x_pos + offsets[1], diffs_pivot["rate_latent_bpp_diff"], bar_width, color=bpp2_color
)

# Plot PSNR bar on ax2
ax2.bar(x_pos + offsets[2], diffs_pivot["psnr_db_diff"], bar_width, color=psnr_color)

# --- 5. Customize the Plot ---
ax1.set_title(
    "Ablation Study: Metric changes with respect to N-O Cool-chic", fontsize=12, pad=10
)
ax1.set_xlabel("Model Variant", fontsize=10)
ax1.set_ylabel("$\\Delta$ bpp ($\\leftarrow$ better)", fontsize=10)
ax2.set_ylabel(
    "$\\Delta$ PSNR (dB) ($\\rightarrow$ better)", fontsize=10, color=psnr_color
)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(variants, fontsize=8)
ax2.tick_params(axis="y", labelcolor=psnr_color)
ax2.spines["right"].set_color(psnr_color)

# --- 6. Align the zero-lines of the two axes (The Fix) ---
# Get the current limits
y1_min, y1_max = ax1.get_ylim()
y2_min, y2_max = ax2.get_ylim()

# Calculate the largest absolute value for each axis
abs_max1 = max(abs(y1_min), abs(y1_max))
abs_max2 = max(abs(y2_min), abs(y2_max))

# Set symmetrical limits on both axes with a 10% margin
ax1.set_ylim(-abs_max1 * 1.1, abs_max1 * 1.1)
ax2.set_ylim(-abs_max2 * 1.1, abs_max2 * 1.1)

# Now that they are aligned, one zero-line is sufficient
ax1.axhline(0, color="black", linewidth=1.0, linestyle="--")


# --- 6. Create a Manual Legend ---
legend_elements = [
    Patch(
        facecolor=bpp1_color, edgecolor="black", label="$\Delta$ Modulation rate (bpp)"
    ),
    Patch(facecolor=bpp2_color, edgecolor="black", label="$\Delta$ Latent rate (bpp)"),
    Patch(facecolor=psnr_color, edgecolor="black", label="$\Delta$ PSNR (dB)"),
]
ax1.legend(
    handles=legend_elements,
    loc="best",
    # bbox_to_anchor=(0.5, 0.15),
    # ncol=3,
    fancybox=True,
    shadow=False,
)

# Adjust axis limits to prevent bars from touching the edges
ax1_min, ax1_max = ax1.get_ylim()
ax1.set_ylim(ax1_min * 1.15, ax1_max * 1.15)
ax2_min, ax2_max = ax2.get_ylim()
ax2.set_ylim(ax2_min * 1.15, ax2_max * 1.15)


# plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the title
plt.tight_layout()  # Adjust layout to make room for the title
plt.show()